In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import librosa
import collections 

from sklearn.neighbors import NearestCentroid
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [2]:
unit_len = 1*22050
sub_len = 10*unit_len
step = 8*unit_len
bands = 20
languages = ['english','hindi','urdu','telugu','punjabi','malayalam','assamese','bengali','marathi','odiya','tamil','kannada']

In [3]:
def feature_normalize(dataset):
    mu = np.mean(dataset, axis=0)
    dataset -= mu 
    maxx = np.max(abs(dataset), axis=0)
    return dataset / maxx

In [4]:
def energy(samples):
    return np.sum(np.power(samples, 2.)) / float(len(samples))

In [5]:
def remove_silence(sound_clip, silence_len=1):
    new_clip = np.asarray([])
    sub_len = int(silence_len*22050)
    for i in range(0, len(sound_clip), sub_len):
        e = energy(sound_clip[i:i+sub_len])
        if e>0.001:
            new_clip=  np.concatenate((new_clip,sound_clip[i:i+sub_len]))
    return new_clip

In [6]:
def CountFrequency(arr): 
    return collections.Counter(arr) 

## Get model

In [7]:
with open("./data-pickle/ncc-classifier-2.pkl","rb") as f:
    classifier = pickle.load(f)

## Get File

In [8]:
file_path = './unknown-data/unknown_wav/telugu-2.wav'
mfccs = []
sound_clip,sr = librosa.load(file_path)
sound_clip = feature_normalize(sound_clip)
sound_clip = remove_silence(sound_clip[unit_len*30:])
for i in range(0, len(sound_clip) - sub_len, step):
    mfcc = librosa.feature.mfcc(y=sound_clip[i: i + sub_len], sr=sr, n_mfcc = bands).T 
    mfccs.append(mfcc)

In [9]:
X = np.asarray(mfccs)
X = X.reshape(len(X),-1)
X.shape

(58, 8620)

In [10]:
y = classifier.predict(X)

In [11]:
result = [languages[i] for i in y]
result

['telugu',
 'telugu',
 'telugu',
 'telugu',
 'telugu',
 'telugu',
 'telugu',
 'telugu',
 'telugu',
 'telugu',
 'malayalam',
 'malayalam',
 'punjabi',
 'telugu',
 'malayalam',
 'malayalam',
 'malayalam',
 'malayalam',
 'malayalam',
 'malayalam',
 'malayalam',
 'malayalam',
 'malayalam',
 'malayalam',
 'malayalam',
 'punjabi',
 'malayalam',
 'telugu',
 'telugu',
 'telugu',
 'malayalam',
 'malayalam',
 'malayalam',
 'malayalam',
 'malayalam',
 'punjabi',
 'malayalam',
 'malayalam',
 'malayalam',
 'telugu',
 'telugu',
 'telugu',
 'telugu',
 'telugu',
 'telugu',
 'telugu',
 'malayalam',
 'telugu',
 'telugu',
 'telugu',
 'telugu',
 'telugu',
 'malayalam',
 'telugu',
 'malayalam',
 'telugu',
 'malayalam',
 'malayalam']

In [13]:
counter = CountFrequency(result) 
print(counter.keys())
print(counter.values())

dict_keys(['telugu', 'malayalam', 'punjabi'])
dict_values([28, 27, 3])
